# Distance Notebook
### Calculate the distance each team will need to travel over the course of the season

#### Note
- ISSUES TO ADDRESS
    - need to check the arena location data and the merge because it seems some of the locations may not be correct for example the Michigan State at Lake State is reporting a distance of double what it should 
    - The travel flag doesnt seem to be working correctly either 
- as of 9-8-24 . THe schedule table doesn't seem complete. Big Ten teams have incomplete schedule
- Does not account for nuetral site and tourny games

In [185]:
# Dependencies
import os
import sys
import pandas as pd
import numpy as np
import geopy
from geopy.distance import geodesic


In [186]:
## Paths to source data
# SCHEDULE FILE
schedule_path = os.path.join('..', 'data', 'schedule', '2024_current.csv')
schedule_data = pd.read_csv(schedule_path)
# ARENA INFO FILE
arena_path = os.path.join('..', 'data', 'arena_school_info_2.csv')
arena_data = pd.read_csv(arena_path)

# NEUTRAL SITE ARENA INFORMATION FILE
neutral_path = os.path.join('..', 'data', 'neutral_arenas_2024.csv')
neutral_arenas_df = pd.read_csv(neutral_path)

# Display data
# schedule_data.head()
# schedule_data.tail()
# arena_data.head()
neutral_arenas_df.tail()

,Flag,Arena_Name,latitude,longitude
12,"Desert Hockey Classic (at Mullett Arena, Tempe...","Mullett Arena, Tempe, Ariz.",33.426519,-111.928551
13,"at Wrigley Field, Chicago, Ill.","Wrigley Field, Chicago, Ill.",41.948343,-87.655537
14,"Connecticut Ice Tournament (at XL Center, Hart...","XL Center, Hartford, Conn.",41.768459,-72.677094
15,MVP Mayor's Cup,"MVP Arena, Albany, N.Y.",42.648594,-73.755018
16,Little Caesars,"Little Caesars Arena, Detroit, Mich",42.341110,-83.055299


## Account for nuetral site games

In [187]:
# Identify rows in the schedule table that involve neutral site games
# by checking if the 'Conference' or 'Game_Notes' contains a match with the 'Flag' in the neutral arenas table.

# Helper function to find if any flag appears in the Conference or Game_Notes columns
def is_neutral_game(row, flags):
    # Check for flags in both Conference and Game_Notes, ensuring correct handling of NaN
    conference_match = any(flag in str(row['Conference']) for flag in flags)
    notes_match = any(flag in str(row['Game_Notes']) for flag in flags)
    return conference_match or notes_match

# Extract the list of flags from the neutral arenas table (assuming already loaded neutral_arenas_df)
neutral_flags = neutral_arenas_df['Flag'].tolist()

# Apply the function to the schedule data to identify neutral site games
schedule_data['Is_Neutral_Game'] = schedule_data.apply(is_neutral_game, axis=1, flags=neutral_flags)

# Filter the schedule for neutral site games
neutral_site_games = schedule_data[schedule_data['Is_Neutral_Game']]

# # Remove Rows that have TBD or a / in one of the team columns
# neutral_site_games = neutral_site_games[~neutral_site_games['Home_Team'].str.contains('/')]
# neutral_site_games = neutral_site_games[~neutral_site_games['Away_Team'].str.contains('/')]
# neutral_site_games = neutral_site_games[~neutral_site_games['Home_Team'].str.contains('TBD')]
# neutral_site_games = neutral_site_games[~neutral_site_games['Away_Team'].str.contains('TBD')]


# Display the neutral site games and info
# neutral_site_games.head()
# neutral_site_games.tail()
# # neutral_site_games.info()
# # neutral_flags
# # Reindex the neutral site games
# neutral_site_games.reset_index(drop=True, inplace=True)
# neutral_site_games

### Calculate the distance to neutral site locations

In [188]:
### refactor the code to include the distance between the two teams


# Helper function to calculate the distance between two points (lat, lon)
def calculate_distance(lat1, lon1, lat2, lon2):
    if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
        return geodesic((lat1, lon1), (lat2, lon2)).miles
    else:
        return None  # Return None if any coordinates are missing

# Function to merge team location data and neutral arenas
def merge_team_and_arena_data(schedule_df, team_df, arena_df):
    # Merge team locations (away and home teams)
    schedule_df = schedule_df.merge(
        team_df[['Team', 'Latitude', 'Longitude']], 
        left_on='Away_Team', right_on='Team', how='left', suffixes=('', '_away')
    )
    schedule_df = schedule_df.merge(
        team_df[['Team', 'Latitude', 'Longitude']], 
        left_on='Home_Team', right_on='Team', how='left', suffixes=('', '_home')
    )

    # Merge on 'Conference' column first
    schedule_df = schedule_df.merge(
        arena_df[['Flag', 'latitude', 'longitude']], 
        left_on='Conference', right_on='Flag', how='left'
    )

    # Handle rows where the Conference merge did not work by checking 'Game_Notes'
    missing_coords_df = schedule_df[schedule_df['latitude'].isnull()].copy()

    def match_flag_in_game_notes(row, arena_df):
        for _, flag_row in arena_df.iterrows():
            if flag_row['Flag'] in str(row['Game_Notes']):
                return flag_row['latitude'], flag_row['longitude']
        return None, None

    # Apply partial matching function for missing coordinates
    missing_coords_df[['latitude', 'longitude']] = missing_coords_df.apply(
        lambda row: match_flag_in_game_notes(row, arena_df), axis=1, result_type="expand"
    )

    # Fill missing latitude/longitude
    schedule_df.loc[schedule_df['latitude'].isnull(), ['latitude', 'longitude']] = missing_coords_df[['latitude', 'longitude']]

    return schedule_df

# # Function to calculate distances to neutral site
# def calculate_team_distances(schedule_df):
#     schedule_df['Away_Distance'] = schedule_df.apply(
#         lambda row: calculate_distance(row['Latitude'], row['Longitude'], row['latitude'], row['longitude']), axis=1
#     )
#     schedule_df['Home_Distance'] = schedule_df.apply(
#         lambda row: calculate_distance(row['Latitude_home'], row['Longitude_home'], row['latitude'], row['longitude']), axis=1
#     )

#     return schedule_df

# # Refactor into steps
# neutral_site_games = merge_team_and_arena_data(schedule_data, team_locations_df, neutral_arenas_df)
# neutral_site_games = calculate_team_distances(neutral_site_games)

# Filter out rows with missing distances
# neutral_site_games = neutral_site_games.dropna(subset=['Away_Distance', 'Home_Distance'])

# Output results
output_path = '../TEMP/neutral_site_games_distances_TEST2.csv'
neutral_site_games.to_csv(output_path, index=False)

## PRINT DF INFOR BEFORE THE CLEANING
# print(neutral_site_games.info())

In [189]:
### CLEANING BASED ON LOOK AT OUTPUT FROM CELL ABOVE


# Remove Rows that have TBD or a / in one of the team columns
neutral_site_games = neutral_site_games[~neutral_site_games['Home_Team'].str.contains('/')]
neutral_site_games = neutral_site_games[~neutral_site_games['Away_Team'].str.contains('/')]
neutral_site_games = neutral_site_games[~neutral_site_games['Home_Team'].str.contains('TBD')]
neutral_site_games = neutral_site_games[~neutral_site_games['Away_Team'].str.contains('TBD')]

# Print DF INFO AFTER CLEANING
# print(neutral_site_games.info())

# OUTPUT CSV FOR CHECKING INTO TEMP FOLDER
output_path = '../TEMP/neutral_site_games_distances_CLEANED2.csv'
neutral_site_games.to_csv(output_path, index=False)

## NOTE - STOPING HERE 
- above is looking OK cleaning / filtering the nuetral games. leaving 30 at the moment (Sunday before watching lions game) the 

In [190]:
neutral_site_games.head()

,Date,Conference,Game_Notes,Away_Team,Away_Team_Link,Away_Score,Home_Team,Home_Team_Link,Home_Score,OT,Box_Link,Metrics_Link,Day,Game_ID,Is_Neutral_Game
46,2024-10-11,"Ice Breaker (at Orleans Arena, Paradise, Nev.)",NaN,Omaha,/reports/team/Omaha/37,NaN,Massachusetts,/reports/team/Massachusetts/27,NaN,NaN,NaN,NaN,Friday,2024-10-11_Massachusetts_Omaha,True
47,2024-10-11,"Ice Breaker (at Orleans Arena, Paradise, Nev.)",NaN,Minnesota,/reports/team/Minnesota/34,NaN,Air Force,/reports/team/Air-Force/1,NaN,NaN,NaN,NaN,Friday,2024-10-11_Air Force_Minnesota,True
89,2024-10-12,Non-Conference,US Hockey Hall of Fame game,Providence,/reports/team/Providence/46,NaN,North Dakota,/reports/team/North-Dakota/40,NaN,NaN,NaN,NaN,Saturday,2024-10-12_North Dakota_Providence,True
202,2024-10-26,Non-Conference,"at Xcel Energy Center, St. Paul, Minn.",Minnesota,/reports/team/Minnesota/34,NaN,St. Thomas,/reports/team/St-Thomas/63,NaN,NaN,NaN,NaN,Saturday,2024-10-26_St. Thomas_Minnesota,True
366,2024-11-16,CCHA,"at Centre Ice Arena, Traverse City, Mich.",Ferris State,/reports/team/Ferris-State/21,NaN,Lake Superior,/reports/team/Lake-Superior/24,NaN,NaN,NaN,NaN,Saturday,2024-11-16_Lake Superior_Ferris State,True


In [191]:
## New Approach - create a new table and structure for the neutral site games - add them to the agg count at the end

## Assign a location to each game - based on the FLag column from neutral_arenas_df

# Drop the columns that are not needed
neutral_site_games = neutral_site_games.drop(columns=['Away_Team_Link', 'Away_Score', 'Home_Team_Link', 'Home_Score', 'OT', 'Box_Link', 'Metrics_Link'])

# Reinex the DF
neutral_site_games.reset_index(drop=True, inplace=True)

# If Game_Notes is NaN fill with Conference
neutral_site_games['Game_Notes'] = neutral_site_games['Game_Notes'].fillna(neutral_site_games['Conference'])

# Assign a location to each game
# Look for Game_Notes that contain the Flag from neutral_arenas_df
# If there is a match, assign the location to the game

# Helper function to assign latitude and longitude to each game
def assign_location(row, arena_df):
    for _, flag_row in arena_df.iterrows():
        if flag_row['Flag'] in str(row['Game_Notes']):
            return flag_row['latitude'], flag_row['longitude']
    return None, None

# Extract the list of flags from the neutral arenas table (assuming already loaded neutral_arenas_df)
neutral_flags = neutral_arenas_df['Flag'].tolist()

# Apply the function to the schedule data to identify neutral site games
neutral_site_games[['latitude', 'longitude']] = neutral_site_games.apply(assign_location, axis=1, arena_df=neutral_arenas_df, result_type="expand")






# neutral_site_games.head()
# neutral_site_games.tail()
# neutral_site_games




## WORKING ABOVE

In [192]:
### Calculate the distance between the two teams for each game

# Helper function to calculate the distance between two points (lat, lon)
def calculate_distance(lat1, lon1, lat2, lon2):
    if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
        return geodesic((lat1, lon1), (lat2, lon2)).miles
    else:
        return None  # Return None if any coordinates are missing

# Function to calculate distances to neutral site
def calculate_team_distances(schedule_df):
    schedule_df['Away_Distance'] = schedule_df.apply(
        lambda row: calculate_distance(row['Latitude'], row['Longitude'], row['latitude'], row['longitude']), axis=1
    )
    schedule_df['Home_Distance'] = schedule_df.apply(
        lambda row: calculate_distance(row['Latitude_home'], row['Longitude_home'], row['latitude'], row['longitude']), axis=1
    )

    return schedule_df

# Refactor into steps
neutral_site_games = merge_team_and_arena_data(schedule_data, arena_data, neutral_arenas_df)
neutral_site_games = calculate_team_distances(neutral_site_games)

# Filter out rows with missing distances
neutral_site_games = neutral_site_games.dropna(subset=['Away_Distance', 'Home_Distance'])


# Output results
output_path = '../TEMP/neutral_site_games_distances_TESTv3.csv'
neutral_site_games.to_csv(output_path, index=False)

neutral_site_games.head()
neutral_site_games.tail()
neutral_site_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, 46 to 939
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             29 non-null     object 
 1   Conference       29 non-null     object 
 2   Game_Notes       12 non-null     object 
 3   Away_Team        29 non-null     object 
 4   Away_Team_Link   29 non-null     object 
 5   Away_Score       0 non-null      float64
 6   Home_Team        29 non-null     object 
 7   Home_Team_Link   29 non-null     object 
 8   Home_Score       0 non-null      float64
 9   OT               0 non-null      float64
 10  Box_Link         0 non-null      float64
 11  Metrics_Link     3 non-null      object 
 12  Day              29 non-null     object 
 13  Game_ID          29 non-null     object 
 14  Is_Neutral_Game  29 non-null     bool   
 15  Team             29 non-null     object 
 16  Latitude         29 non-null     float64
 17  Longitude        29 n

In [193]:
### DESIRED OUTPUT
# Date, Game_ID, Game_Notes, Team, Distance for each team in each game

# Create a new DataFrame to store the results
neutral_site_games_agg = pd.DataFrame(columns=['Date', 'Game_ID', 'Game_Notes', 'Team', 'N_Distance'])

# Iterate through the neutral_site_games DataFrame and add the data to the new DataFrame
rows = []  # Use a list to accumulate rows for better performance
for index, row in neutral_site_games.iterrows():
    # Add the Away Team data
    rows.append({
        'Date': row['Date'],
        'Game_ID': row['Game_ID'],
        'Game_Notes': row['Game_Notes'],
        'Team': row['Away_Team'],
        'N_Distance': row['Away_Distance']
    })
    
    # Add the Home Team data
    rows.append({
        'Date': row['Date'],
        'Game_ID': row['Game_ID'],
        'Game_Notes': row['Game_Notes'],
        'Team': row['Home_Team'],
        'N_Distance': row['Home_Distance']
    })

# Convert the list of rows into a DataFrame and concatenate
neutral_site_games_agg = pd.concat([neutral_site_games_agg, pd.DataFrame(rows)], ignore_index=True)

# Output results
# output_path = '../TEMP/neutral_site_games_distances_agg_TESTv4.csv'
# neutral_site_games_agg.to_csv(output_path, index=False)

# Display the first and last few rows of the DataFrame
neutral_site_games_agg.head()
# neutral_site_games_agg.tail()

C:\Users\jbanc\AppData\Local\Temp\ipykernel_6184\318824665.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  neutral_site_games_agg = pd.concat([neutral_site_games_agg, pd.DataFrame(rows)], ignore_index=True)


,Date,Game_ID,Game_Notes,Team,N_Distance
0,2024-10-11,2024-10-11_Massachusetts_Omaha,NaN,Omaha,1093.593499
1,2024-10-11,2024-10-11_Massachusetts_Omaha,NaN,Massachusetts,2383.665482
2,2024-10-11,2024-10-11_Air Force_Minnesota,NaN,Minnesota,1302.725571
3,2024-10-11,2024-10-11_Air Force_Minnesota,NaN,Air Force,600.521127
4,2024-10-12,2024-10-12_North Dakota_Providence,US Hockey Hall of Fame game,Providence,1319.541996


## Continued on Sunday Above

### Merge the arena info into schedule table 
## NON NUETRAL GAMES

In [194]:
# Merge the schedule data with the arena data to include home and away team locations

# First, ensure team names match between datasets
# We will merge on the 'Team' column in the arena data and 'Home_Team'/'Away_Team' in the schedule data
merged_data = schedule_data.merge(arena_data[['Team', 'Latitude', 'Longitude']], left_on='Home_Team', right_on='Team', how='left')
merged_data = merged_data.rename(columns={'Latitude': 'Home_Latitude', 'Longitude': 'Home_Longitude'})

# Merge again for the away teams
merged_data = merged_data.merge(arena_data[['Team', 'Latitude', 'Longitude']], left_on='Away_Team', right_on='Team', how='left')
merged_data = merged_data.rename(columns={'Latitude': 'Away_Latitude', 'Longitude': 'Away_Longitude'})

# Drop the unnecessary 'Team' columns from the merged data
merged_data = merged_data.drop(columns=['Team_x', 'Team_y'])

# Display the first few rows of the merged data to verify the result
merged_data.head()

,Date,Conference,Game_Notes,Away_Team,Away_Team_Link,Away_Score,Home_Team,Home_Team_Link,Home_Score,OT,Box_Link,Metrics_Link,Day,Game_ID,Is_Neutral_Game,Home_Latitude,Home_Longitude,Away_Latitude,Away_Longitude
0,2024-10-04,Non-Conference,NaN,Michigan State,/reports/team/Michigan-State/32,NaN,Lake Superior,/reports/team/Lake-Superior/24,NaN,NaN,NaN,NaN,Friday,2024-10-04_Lake Superior_Michigan State,False,46.784622,-92.144841,42.728266,-84.489366
1,2024-10-04,Non-Conference,NaN,Minnesota State,/reports/team/Minnesota-State/35,NaN,Michigan,/reports/team/Michigan/31,NaN,NaN,NaN,NaN,Friday,2024-10-04_Michigan_Minnesota State,False,42.254222,-83.778199,44.144307,-93.995043
2,2024-10-04,Non-Conference,NaN,Arizona State,/reports/team/Arizona-State/61,NaN,Air Force,/reports/team/Air-Force/1,NaN,NaN,NaN,NaN,Friday,2024-10-04_Air Force_Arizona State,False,39.013739,-104.883727,33.447156,-111.910867
3,2024-10-05,Non-Conference,NaN,Providence,/reports/team/Providence/46,NaN,Union,/reports/team/Union/54,NaN,NaN,NaN,NaN,Saturday,2024-10-05_Union_Providence,False,42.818004,-73.924824,41.844005,-71.434748
4,2024-10-05,Non-Conference,NaN,Holy Cross,/reports/team/Holy-Cross/23,NaN,Boston University,/reports/team/Boston-University/10,NaN,NaN,NaN,NaN,Saturday,2024-10-05_Boston University_Holy Cross,False,42.353838,-71.120653,42.239239,-71.807961


### Caculate the distance between each school
- Using Haversine equation to calculate the straight line distance between two sets of lat/lon coodinates

In [195]:
# Function to calculate the haversine distance (numpy) between two points (lat1, lon1) and (lat2, lon2)
def haversine(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    # Radius of Earth in miles
    r = 3956
    return c * r

# Apply the Haversine function to each row to calculate the distance between the home and away arenas
merged_data['Distance_Miles'] = merged_data.apply(
    lambda row: haversine(row['Home_Latitude'], row['Home_Longitude'], row['Away_Latitude'], row['Away_Longitude']),
    axis=1
)

# Display the updated data with the calculated distance
merged_data[['Home_Team', 'Away_Team', 'Distance_Miles']].head()

,Home_Team,Away_Team,Distance_Miles
0,Lake Superior,Michigan State,467.960246
1,Michigan,Minnesota State,530.146744
2,Air Force,Arizona State,548.193384
3,Union,Providence,143.786679
4,Boston University,Holy Cross,35.982031


### Filter out results to avoid double counting games on weekend series

- Travel_Flag to account for consecutive games played at the same venue within a 3-day span. If a team plays multiple games at the same venue within this period, travel is only counted for the first game.

- The Adjusted_Travel_Distance column reflects the distance a team will travel for each game, considering the consecutive game rule.

In [196]:
### VERSION 2
# Update logic to handle non-consecutive rows by grouping first

# Convert 'Date' column to datetime format for easier manipulation
merged_data['Date'] = pd.to_datetime(merged_data['Date'])

# Reset the travel flag
merged_data['Travel_Flag'] = 1

# Sort the data by 'Away_Team', 'Home_Team', and 'Date' to ensure games are grouped correctly
merged_data = merged_data.sort_values(by=['Away_Team', 'Home_Team', 'Date'])

# Group by 'Away_Team' and 'Home_Team', then iterate through each group to set the travel flag
for (away_team, home_team), group in merged_data.groupby(['Away_Team', 'Home_Team']):
    group = group.sort_values(by='Date')  # Sort by date within each group
    
    # Iterate through the group to check for consecutive games
    for i in range(1, len(group)):
        current_game = group.iloc[i]
        previous_game = group.iloc[i - 1]
        
        # Check if the games are within 3 days
        if (current_game['Date'] - previous_game['Date']).days <= 3:
            # Set the travel flag to 0 for the current game
            merged_data.loc[current_game.name, 'Travel_Flag'] = 0

# Only consider rows where travel flag is 1 for calculating total travel distance
merged_data['Adjusted_Travel_Distance'] = merged_data['Distance_Miles'] * merged_data['Travel_Flag']

# Display the updated data with the travel flag and adjusted distance
merged_data[['Away_Team', 'Home_Team', 'Date', 'Distance_Miles', 'Travel_Flag', 'Adjusted_Travel_Distance']].head()

# OUTPUT TABLE TO TEMP FILE FOR TESTING
output_path = os.path.join('..', 'TEMP', 'distance_test_v2.csv')
merged_data.to_csv(output_path, index=False)

In [197]:
### ORIGINAL CODE FOR TRAVEL DISTANCE CALCULATION

# # Convert 'Date' column to datetime format for easier manipulation
# merged_data['Date'] = pd.to_datetime(merged_data['Date'])

# # Sort the data by 'Away_Team' and 'Date' to identify consecutive games at the same venue
# merged_data = merged_data.sort_values(by=['Away_Team', 'Date'])

# # Initialize a flag column to indicate whether the travel should be counted (1 = yes, 0 = no)
# merged_data['Travel_Flag'] = 1

# # Iterate through the rows and check for consecutive games at the same venue within a 3-day span
# for i in range(1, len(merged_data)):
#     current_game = merged_data.iloc[i]
#     previous_game = merged_data.iloc[i - 1]
    
#     # Check if the away team is the same, and the venue (home team) is the same, and the games are within 3 days
#     if (current_game['Away_Team'] == previous_game['Away_Team'] and
#         current_game['Home_Team'] == previous_game['Home_Team'] and
#         (current_game['Date'] - previous_game['Date']).days <= 3):
#         # Set the travel flag to 0 for the current game (no additional travel)
#         merged_data.at[i, 'Travel_Flag'] = 0

# # Only consider rows where travel flag is 1 for calculating total travel distance
# merged_data['Adjusted_Travel_Distance'] = merged_data['Distance_Miles'] * merged_data['Travel_Flag']

# # Display the updated data with the travel flag and adjusted distance
# merged_data[['Away_Team', 'Home_Team', 'Date', 'Distance_Miles', 'Travel_Flag', 'Adjusted_Travel_Distance']].head()


## Aggregate Total Travel Distance for Each Team AND
## Calculate and store the Trip Count and the Average trip distance

In [198]:
# Only consider rows where travel flag is 1 for calculating total travel distance
merged_data['Adjusted_Travel_Distance'] = merged_data['Distance_Miles'] * merged_data['Travel_Flag']

# Calculate the total travel distance per team
team_travel_distances = merged_data.groupby('Away_Team')['Adjusted_Travel_Distance'].sum().reset_index()
team_travel_distances.columns = ['Team', 'Total_Travel_Distance']

# Step 4: Adding Trip Count and Average Trip Distance

# Calculate the number of trips for each team
trip_count = merged_data[merged_data['Travel_Flag'] == 1].groupby('Away_Team').size().reset_index(name='Trip_Count')

# Merge trip count with travel distances
team_travel_data = pd.merge(team_travel_distances, trip_count, left_on='Team', right_on='Away_Team', how='left').drop(columns='Away_Team')

# Calculate average trip distance
team_travel_data['Average_Trip_Distance'] = team_travel_data['Total_Travel_Distance'] / team_travel_data['Trip_Count']

# Display the top 5 teams with the highest average trip distance
team_travel_data = team_travel_data.sort_values(by='Average_Trip_Distance', ascending=False)
team_travel_data.head()

,Team,Total_Travel_Distance,Trip_Count,Average_Trip_Distance
2,Alaska-Anchorage,30663.509981,12,2555.292498
1,Alaska,28413.140021,12,2367.761668
4,Arizona State,10730.958743,9,1192.328749
0,Air Force,10221.145428,9,1135.682825
25,Denver,10233.783848,10,1023.378385


## Add the Calculations for Nuetral Site Games to finished table

- NOT SURE IF THIS IS ADDING BOTH DISTANCES IF TEAM PLAYS 2 or MORTE NUET SITE GAMES (FOR EXAMPLE MICHIGAN STATE - We Have 3)

In [199]:
## If a team has a game at a neutral site, the distance from that table is added to the total distance for that team and the trip count is incremented by 1.

# Merge the neutral site games data with the team travel data to update the total travel distance and trip count
team_travel_data = team_travel_data.merge(neutral_site_games_agg, left_on='Team', right_on='Team', how='left')

# Fill missing values with 0 for teams that did not have neutral site games
team_travel_data['N_Distance'] = team_travel_data['N_Distance'].fillna(0)

# Update the total travel distance and trip count based on neutral site games
team_travel_data['Total_Travel_Distance'] += team_travel_data['N_Distance']
team_travel_data['Trip_Count'] += team_travel_data['N_Distance'].apply(lambda x: 1 if x > 0 else 0)

# Recalculate the average trip distance
team_travel_data['Average_Trip_Distance'] = team_travel_data['Total_Travel_Distance'] / team_travel_data['Trip_Count']

# Display the top 5 teams with the highest average trip distance
team_travel_data = team_travel_data.sort_values(by='Average_Trip_Distance', ascending=False)
team_travel_data.head()


,Team,Total_Travel_Distance,Trip_Count,Average_Trip_Distance,Date,Game_ID,Game_Notes,N_Distance
0,Alaska-Anchorage,30663.509981,12,2555.292498,NaN,NaN,NaN,0.000000
1,Alaska,31197.355610,13,2399.796585,2024-12-28,2024-12-28_Alaska_Connecticut,NaN,2784.215589
3,Air Force,10821.666556,10,1082.166656,2024-10-11,2024-10-11_Air Force_Minnesota,NaN,600.521127
2,Arizona State,10732.710008,10,1073.271001,2025-01-03,2025-01-03_Arizona State_Robert Morris,NaN,1.751265
4,Denver,10233.783848,10,1023.378385,NaN,NaN,NaN,0.000000


In [200]:
## CLEANING HOTFIX - REMOVE DUPLICATE COLUMNS
## IF a Team appears twice keep the row with the larger Total_Travel_Distance
team_travel_data = team_travel_data.sort_values(by='Total_Travel_Distance', ascending=False)
team_travel_data = team_travel_data.drop_duplicates(subset='Team', keep='first')

# Drop rows with 0 Total_Travel_Distance
team_travel_data = team_travel_data[team_travel_data['Total_Travel_Distance'] > 0]



## Find The Closest Other Team to Each team and Store

In [201]:
from geopy.distance import geodesic
import pandas as pd

# Helper function to calculate the distance between two points (lat, lon)
def calculate_distance(lat1, lon1, lat2, lon2):
    if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
        return geodesic((lat1, lon1), (lat2, lon2)).miles
    else:
        return None  # Return None if any coordinates are missing

# Function to find the closest team to a specific team in the arena data
def find_closest_team(current_team_row, team_data):
    # Initialize variables to store the closest team and distance
    closest_team = None
    closest_distance = np.inf
    
    # Iterate over each row in the team data
    for _, row in team_data.iterrows():
        # Calculate the distance between the current team and the other team
        distance = calculate_distance(current_team_row['Latitude'], current_team_row['Longitude'], row['Latitude'], row['Longitude'])
        
        # Update the closest team if the distance is smaller
        if distance is not None and distance < closest_distance:
            closest_team = row['Team']
            closest_distance = distance
    
    return closest_team, closest_distance

# Find the closest team to each team in the arena data
closest_teams = []
closest_distances = []

# Iterate over each row in the arena data to find the closest team
for _, row in arena_data.iterrows():
    # Exclude the current team from the comparison
    other_teams = arena_data[arena_data['Team'] != row['Team']]
    closest_team, closest_distance = find_closest_team(row, other_teams)
    
    closest_teams.append(closest_team)
    closest_distances.append(closest_distance)

# Add the closest team and distance to the arena data
arena_data['Closest_Team'] = closest_teams
arena_data['Closest_Distance'] = closest_distances

# Merge the closest team data with the team travel data
team_travel_data = team_travel_data.merge(arena_data[['Team', 'Closest_Team', 'Closest_Distance']], on='Team', how='left')

# Sort by closest distance to another team
team_travel_data = team_travel_data.sort_values(by='Closest_Distance')

# Display the top 5 teams with the highest average trip distance and closest team information
team_travel_data.head(20)

,Team,Total_Travel_Distance,Trip_Count,Average_Trip_Distance,Date,Game_ID,Game_Notes,N_Distance,Closest_Team,Closest_Distance
46,Michigan State,2966.556550,12,247.213046,2025-01-04,2025-01-04_Wisconsin_Michigan State,"at Wrigley Field, Chicago, Ill.",170.821767,Long Island,0.567858
27,Long Island,4709.334130,9,523.259348,NaN,NaN,NaN,0.000000,Michigan State,0.567858
24,Boston University,5043.483789,14,360.248842,2024-11-29,2024-11-29_Boston University_Merrimack,NaN,2994.665589,Harvard,0.940023
14,Harvard,5671.903426,19,298.521233,2024-11-29,2024-11-29_Notre Dame_Harvard,NaN,2994.249148,Boston University,0.940023
50,Northeastern,2787.652589,18,154.869588,2024-12-28,2024-12-28_Providence_Northeastern,NaN,111.693419,Boston University,1.939860
59,Brown,1939.565471,13,149.197344,NaN,NaN,NaN,0.000000,Providence,2.043651
32,Providence,3728.451495,16,233.028218,2024-10-12,2024-10-12_North Dakota_Providence,US Hockey Hall of Fame game,1319.541996,Brown,2.043651
54,Boston College,2327.687331,13,179.052872,NaN,NaN,NaN,0.000000,Boston University,2.755747
25,Minnesota-Duluth,4859.558904,8,607.444863,NaN,NaN,NaN,0.000000,Lake Superior,3.715345
17,Lake Superior,5458.724736,11,496.247703,2024-11-16,2024-11-16_Lake Superior_Ferris State,"at Centre Ice Arena, Traverse City, Mich.",349.711653,Minnesota-Duluth,3.715345


In [202]:
# ## Find The closest other team to each team in the areana data

# # Helper function to calculate the distance between two points (lat, lon)
# def calculate_distance(lat1, lon1, lat2, lon2):
#     if pd.notnull(lat1) and pd.notnull(lon1) and pd.notnull(lat2) and pd.notnull(lon2):
#         return geodesic((lat1, lon1), (lat2, lon2)).miles
#     else:
#         return None  # Return None if any coordinates are missing

# # Function to find the closest team to each team in the arena data
# def find_closest_team(team_data):
#     # Initialize variables to store the closest team and distance
#     closest_team = None
#     closest_distance = np.inf
    
#     # Iterate over each row in the team data
#     for _, row in team_data.iterrows():
#         # Calculate the distance to the current team
#         distance = calculate_distance(team_data['Latitude'], team_data['Longitude'], row['Latitude'], row['Longitude'])
        
#         # Update the closest team if the distance is smaller
#         if distance is not None and distance < closest_distance:
#             closest_team = row['Team']
#             closest_distance = distance
    
#     return closest_team, closest_distance

# # Find the closest team to each team in the arena data
# closest_teams = []
# closest_distances = []

# # Iterate over each row in the arena data to find the closest team
# for _, row in arena_data.iterrows():
#     closest_team, closest_distance = find_closest_team(arena_data.drop(arena_data[arena_data['Team'] == row['Team']].index))
#     closest_teams.append(closest_team)
#     closest_distances.append(closest_distance)

# # Add the closest team and distance to the arena data
# arena_data['Closest_Team'] = closest_teams
# arena_data['Closest_Distance'] = closest_distances

# # Merge the closest team data with the team travel data
# team_travel_data = team_travel_data.merge(arena_data[['Team', 'Closest_Team', 'Closest_Distance']], on='Team', how='left')

# # Display the top 5 teams with the highest average trip distance and closest team information
# team_travel_data.head()



## Output the aggrigate as a csv file (Version 1)
- does not account for nuetral site games, mostly games in tournaments. These games often have teams listed like this 'WMU/Michigan State' or 'Bc/BU' for example

In [203]:
# Output into the TEMP folder
output_path = os.path.join('..', 'TEMP', 'team_travel_distances_v3.csv')
team_travel_data.to_csv(output_path, index=False)
print(f"Output saved to: {output_path}")

PermissionError: [Errno 13] Permission denied: '..\\TEMP\\team_travel_distances_v3.csv'